In [266]:
import requests
from datetime import date, timedelta, datetime
import time
import sys
import statsapi
from pprint import pprint
import json
import os
import csv
# from google.cloud import bigquery



In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser("~/Documents/jupyter/MLB/credentials/mlb-analysis-463501-869e729270f3.json")

client = bigquery.Client()
print("Project:", client.project)

Project: mlb-analysis-463501


In [ ]:
#constant values
mlb = 104
currentSeason = 2025

In [142]:
games = []
seen_games = set()

In [78]:
#Helper function for team stats by season
def team_stat_data(team, group, season, stats='season'):
    url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats"
    params = {
        "season": season,
        "group": group,
        "stats": stats
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json()["stats"][0]["splits"][0]

In [ ]:
# # Get today's schedule
today = date.today()
yesterday = today - timedelta(days = 1)
yesterday = yesterday.isoformat()
schedule = statsapi.schedule(start_date=yesterday, end_date=yesterday, sportId=1, leagueId=104)


#Pull only attribute in list into new array and add relevant fields
#gamePks = [game['game_id'] for game in schedule]

#Pull available game data from schedule
#Each new dictionary counts as one row 
for game in schedule:
    gamePk = game['game_id']
    if gamePk in seen_games:
        continue
    games.append({
        "gamePk": game['game_id'],
        "gametime": game['game_datetime'],
        "awayId": game['away_id'],
        "homeId": game['home_id']
    })
    seen_games.add(game['game_id'])
    
    #Get weather
        boxscore = statsapi.get("game_boxscore", {'gamePk': gamePk})
        for item in boxscore['info']:
            if item['label'] == "Weather":
                games.append({
                    "gamePk": game_pk,
                    "weather": item['value']
                })
                    
        pprint(games)
        sys.exit()
    
# games_dict = {g['gamePk']: g for g in games}
# games = list(games_dict.values())

pprint(games)

In [214]:
#create empty arrays for storing tables
team_game_stats = []
games = []
player_game_stats = []

In [174]:
teams = {}
players = {}

In [215]:
print(boxscore.keys())

dict_keys(['copyright', 'teams', 'officials', 'info', 'pitchingNotes', 'topPerformers'])


In [ ]:
team_game_stats = []
player_game_stats = []
games_today = []

#Get team stats for games today and player stats by game
team_game_stats.append("attribute, statistic, value, gamePk, playerId") #Header row
for g in games:
    gamePk = g['gamePk']
    boxscore = statsapi.get("game_boxscore", {'gamePk': gamePk})
    for side in ["home", "away"]:
        #Add game stats for each team in todays games
        team_stats = boxscore["teams"][side]["teamStats"]
        # Create data structure {team_game_stats} to hold game data, team statistics and player statistics by season and game
        #Batting stats (KAV + Game structure)
        for stat, val in team_stats['batting'].items():
            team_game_stats.append(f"stat_team_batting, {stat}, {val}, {gamePk},")
        for stat, val in away_stats['pitching'].items():
            team_game_stats.append(f"stat_team_pitching, {stat}, {val}, {gamePk},")
        for stat, val in away_stats['fielding'].items():
            team_game_stats.append(f"stat_team_fielding, {stat}, {val}, {gamePk},")
        print(f"Loaded {side} team data from game: {boxscore['teams']['away']['team']['abbreviation']} vs. {boxscore['teams']['home']['team']['abbreviation']}")
        #Player stats (KAV + game + player ID) only if they played any positions
        for player_key, player_data in boxscore["teams"][side]["players"].items():
            player_id = player_data["person"]["id"]  # ✅ This is the integer ID
            url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats"
            params = {"gamePk": gamePk, "stats": "gameLog"}
            response = requests.get(url, params=params)
            player_stats = response.json()
            player_name = player_data['person']['fullName']
            player_stats = player_stats['stats'][0]['splits'][0]
            if('positionsPlayed' not in player_stats):
                continue
            if any(pos['code'] == 'P' for pos in player_stats['positionsPlayed']):
                for stat, val in player_stats['stat']:
                    player_game_stats.append(f"stat_player_pitching, {stat}, {val}, {gamePk}, {player_id}")
                    sys.exit()
            else:
                for stat_val in player_stats['stat']:
                    player_game_stats.append(f"stat_player_hitting, {stat}, {val}, {gamePk}, {player_id}")
            print(f"Loaded player data for {player_name} in game: {boxscore['teams']['away']['team']['abbreviation']} vs. {boxscore['teams']['home']['team']['abbreviation']}")
            # sys.exit()
            

print(player_game_stats[0])
sys.exit()
with open("player_game_stats.csv", mode="w", newline="", encoding="utf-8") as file:
    # Step 2: Create writer with header
    writer = csv.DictWriter(file, fieldnames=player_game_stats[0].keys())

    writer.writeheader()  # Step 3: Write header
    writer.writerows(player_game_stats)  # Step 4: Write rows     

Loaded home team data from game: STL vs. PIT
Loaded player data for Alexander Canario in game: STL vs. PIT
Loaded player data for Adam Frazier in game: STL vs. PIT
Loaded player data for Spencer Horwitz in game: STL vs. PIT
Loaded player data for Bryan Reynolds in game: STL vs. PIT
Loaded player data for Joey Bart in game: STL vs. PIT
Loaded player data for Oneil Cruz in game: STL vs. PIT
Loaded player data for Ke'Bryan Hayes in game: STL vs. PIT
Loaded player data for Jared Triolo in game: STL vs. PIT
Loaded player data for Tommy Pham in game: STL vs. PIT
Loaded player data for Andrew McCutchen in game: STL vs. PIT
Loaded player data for Isiah Kiner-Falefa in game: STL vs. PIT
Loaded player data for Henry Davis in game: STL vs. PIT
Loaded player data for Nick Gonzales in game: STL vs. PIT
Loaded away team data from game: STL vs. PIT
Loaded player data for Nolan Gorman in game: STL vs. PIT
Loaded player data for Victor Scott II in game: STL vs. PIT
Loaded player data for Willson Contre